In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
import random
import gc
from sklearn.model_selection import train_test_split
from sklearn import  metrics
import time
import progressbar
import warnings
import math
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV as LogReg

               
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from WOE_Houston import *

In [2]:
fy_10 = pd.read_csv('fy_10_augmented.csv')
fy_11 = pd.read_csv('fy_11_augmented.csv')
fy_12 = pd.read_csv('fy_12_augmented.csv')
fy_13 = pd.read_csv('fy_13_augmented.csv')
fy_14 = pd.read_csv('fy_14_augmented.csv')
fy_15 = pd.read_csv('fy_15_augmented.csv')
fy_16 = pd.read_csv('fy_16_augmented.csv')

fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15, fy_16]

del fy_10
del fy_11
del fy_12
del fy_13
del fy_14
del fy_15
del fy_16
gc.collect()

essentials = ['Unnamed: 0',
 'Fund Id',
 'Business Area',
 'Fund Center Id',
 'GL Account',
 'GL Description',
 'GL Category',
 'GL Category Description',
 'Revenue or Expenditure',
 'Fiscal Year',
 'Fund Name',
 'Fund Type Descr',
 'Business Area Name',
 'Fund Center Name',
 'Commitment Set Description',
 'Original Budget',
 'Current Budget',
 'Actuals',
 '3Y_Past_Actuals',
 '2Y_Past_Actuals',
 '1Y_Past_Actuals',
 '2Y_1Y_Actuals_Mean',
 '3Y_2Y_1Y_Actuals_Mean',
 '3Y_Past_Original_Budget',
 '2Y_Past_Original_Budget',
 '1Y_Past_Original_Budget',
 '2Y_1Y_Original_Budget_Mean',
 '3Y_2Y_1Y_Original_Budget_Mean',
 '3Y_Past_OA_Difference',
 '2Y_Past_OA_Difference',
 '1Y_Past_OA_Difference',
 '2Y_1Y_OA_Difference_Mean',
 '3Y_2Y_1Y_OA_Difference_Mean',
 'OA_Difference',
 'Yearly Crime Index',
 'Yearly Avg High Temp',
 'Yearly Avg Low Temp',
 'Yearly Rain Fall',
 'Astros Playoff Rounds',
 'Astros Yearly Attendance',
 'Rockets Playoff Rounds',
 'Texans Playoff Rounds',
 'Texans Average Attendance',
 'Rockets Total Attendance']

dummies = ['Fund Name_Asset Forfeiture Fund - Justice',
 'Fund Name_Asset Forfeiture Fund - State',
 'Fund Name_Auto Dealers Fund',
 'Fund Name_Building Inspection Fund',
 'Fund Name_C&E-Civic Center Facility Revenue Fund',
 'Fund Name_Cable Television Fund',
 'Fund Name_Child Safety Fund',
 'Fund Name_Digital Houston Special Revenue',
 'Fund Name_Expansion of Recycling Program',
 'Fund Name_Fleet Management Fund',
 'Fund Name_GSD - In-House Renovation Revolving Fund',
 'Fund Name_General Fund',
 'Fund Name_Greater Houston Trans & Emer Center',
 'Fund Name_HAS-Revenue Fund',
 'Fund Name_Health Benefits Fund',
 'Fund Name_Historic Preservation Fund',
 'Fund Name_Houston Emergeny Center',
 'Fund Name_Long-Term Disablility Plan Fund',
 'Fund Name_Municipal Court Technology Fund',
 'Fund Name_PWE-Combined Utility System Gen Pur Fund',
 'Fund Name_PWE-Combined Utility System Oper Fund',
 'Fund Name_PWE-W & S System Operating Fund',
 'Fund Name_Park Special Revenue Fund',
 'Fund Name_Parking Mangement Operating Fund',
 'Fund Name_Police Special Services',
 'Fund Name_Project Cost Recovery',
 'Fund Name_Property & Casualty Fund',
 'Fund Name_Stormwater Fund',
 'Fund Name_Supplemental Environmental Proj',
 'Fund Name_Workers Compensation Admin Fund',
 'Fund Type Descr_Enterprise Funds',
 'Fund Type Descr_General Funds',
 'Fund Type Descr_Internal Services Funds',
 'Fund Type Descr_Special Revenue Funds',
 'Business Area Name_Admn. & Regulatory Affairs',
 'Business Area Name_Business Opportunity',
 "Business Area Name_City Controller's Office",
 'Business Area Name_City Council',
 'Business Area Name_City Secretary',
 'Business Area Name_Citywide General Government',
 'Business Area Name_Convention and Entertainment',
 'Business Area Name_Finance',
 'Business Area Name_General Debt Services',
 'Business Area Name_General Services Department',
 'Business Area Name_Health & Human Services',
 'Business Area Name_Housing and Community Devp.',
 'Business Area Name_Houston Airport System (HAS)',
 'Business Area Name_Houston Emergency Center (HEC)',
 'Business Area Name_Houston Fire Department (HFD)',
 'Business Area Name_Houston Police Department',
 'Business Area Name_Human Resources Dept.',
 'Business Area Name_Information Technology (IT)',
 'Business Area Name_Legal Department',
 'Business Area Name_Library',
 "Business Area Name_Mayor's Office",
 'Business Area Name_Municipal Court Judicial',
 'Business Area Name_Municipal Courts Department',
 'Business Area Name_Parks & Recreation',
 'Business Area Name_Planning & Development',
 'Business Area Name_Public Works & Engineering-PWE',
 'Business Area Name_Solid Waste Management',
 'Commitment Set Description_50 - Total Expenditures',
 'Business Area Name_Houston Police Department-HPD',
 'Business Area Name_Houston Information Tech Svcs'
]

drop_list = [
    
    'Fund Name', 
    'Fund Type Descr', 
    'Business Area Name',
    'Commitment Set Description',
    'GL Description',
    'GL Category Description',
    'Revenue or Expenditure',
    'Fund Type Descr',
    'Fund Center Name'
]


for i, y in enumerate(fy_list):
    fy_list[i] = pd.concat([y[essentials], y[dummies]], axis=1)
    fy_list[i]['Over Budget'] = fy_list[i]['OA_Difference']>0
    
for i, y in enumerate(fy_list):
    if i==0:
        fy_list[i]["1YP_OA_Diff_Over_80K"]=999999999999999
    else:
        fy_list[i]["1YP_OA_Diff_Over_80K"]=0
        over80k_index = fy_list[i].loc[(fy_list[i]['1Y_Past_OA_Difference'].apply(lambda x: abs(x))>80000.0)].index
        fy_list[i].loc[over80k_index, '1YP_OA_Diff_Over_80K'] = 1
        

        
        
splits_to_try = [
    
                #predicting off "two years"
                 [(0,1),(1,2),'10-11 to predict 11'],
                 [(0,2),(2,3),'10-11 to predict 12'],
                 [(0,3),(3,4),'10-12 to predict 13'],
                 #[(0,3),(3,5),'10-12 to predict 13 and 14'],
                 [(1,4),(4,5),'11-13 to predict 14'], 
                 #[(1,4),(4,6),'11-13 to predict 14 and 15'],
                 [(2,5),(5,6),'12-14 to predict 15'],
                 #[(2,5),(5,7),'12-14 to predict 15 and 16'],
                 [(3,6),(6,7),'13-15 to predict 16'], 
    
                #predicting off "three" years
    
                 [(0,4),(5,6),'10-13 to predict 14'],
                 #[(0,4),(4,6),'10-13 to predict 14 and 15'],
                 #[(0,4),(4,7),'10-13 to predict 14 and 15 and 16'],
                 [(1,5),(5,6),'11-14 to predict 15'],
                 #[(1,5),(5,7),'11-14 to predict 15 and 16'],
                 [(2,6),(6,7),'12-15 to predict 16']
                ]
        


In [3]:
fy_list[0].head()

,Unnamed: 0,Fund Id,Business Area,Fund Center Id,GL Account,GL Description,GL Category,GL Category Description,Revenue or Expenditure,Fiscal Year,Fund Name,Fund Type Descr,Business Area Name,Fund Center Name,Commitment Set Description,Original Budget,Current Budget,Actuals,3Y_Past_Actuals,2Y_Past_Actuals,1Y_Past_Actuals,2Y_1Y_Actuals_Mean,3Y_2Y_1Y_Actuals_Mean,3Y_Past_Original_Budget,2Y_Past_Original_Budget,1Y_Past_Original_Budget,2Y_1Y_Original_Budget_Mean,3Y_2Y_1Y_Original_Budget_Mean,3Y_Past_OA_Difference,2Y_Past_OA_Difference,1Y_Past_OA_Difference,2Y_1Y_OA_Difference_Mean,3Y_2Y_1Y_OA_Difference_Mean,OA_Difference,Yearly Crime Index,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Rain Fall,Astros Playoff Rounds,Astros Yearly Attendance,Rockets Playoff Rounds,Texans Playoff Rounds,Texans Average Attendance,Rockets Total Attendance,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Houston Police Department,Business Area Name_Human Resources Dept.,Business Area Name_Information Technology (IT),Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Court Judicial,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Business Area Name_Houston Police Department-HPD,Business Area Name_Houston Information Tech Svcs,Over Budget,1YP_OA_Diff_Over_80K
0,0,1000,3800,3800050001,501070,501070 - Pension - Civilian,500,500 - Personnel Services,Expenditures,2010,General Fund,General Funds,Health & Human Services,HHS-Office Prog Supp,50 - Total Expenditures,113579.0,113579.0,121344.82,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,7765.82,596.6,80.0,62.0,47.01,0.0,2521076.0,0.0,0.0,71080.0,699280.0,0,0,0,0,0,0,0,0,0,0,0,1

### Information Value

In [4]:
train_iv_list = []
for i, y in enumerate(fy_list):
    train_iv, train_IV_series = data_vars(fy_list[i].drop(['OA_Difference','Actuals','Over Budget','Current Budget'], axis=1),  fy_list[i]['Over Budget'])
    train_iv_list.append([train_iv, train_IV_series])



/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:83: RuntimeWarning: divide by zero encountered in log
  d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:84: RuntimeWarning: divide by zero encountered in log
  d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:83: RuntimeWarning: divide by zero encountered in log
  d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:84: RuntimeWarning: divide by zero encountered in log
  d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:83: RuntimeWarning: divide by zero encountered in log
  d3["WO

/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:84: RuntimeWarning: divide by zero encountered in log
  d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:83: RuntimeWarning: divide by zero encountered in log
  d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:84: RuntimeWarning: divide by zero encountered in log
  d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)


In [5]:
train_iv_list[2][1].sort_values(ascending=False, by='IV').loc[train_iv_list[2][1]['STRENGTH']!='useless']



,VAR_NAME,IV,STRENGTH
53,Fund Center Name,0.547051,suspicious
2,1Y_Past_OA_Difference,0.367461,strong
94,GL Description,0.331271,strong
5,2Y_1Y_OA_Difference_Mean,0.168436,medium
1,1Y_Past_Actuals,0.111936,medium
4,2Y_1Y_Actuals_Mean,0.095565,weak
55,Fund Name,0.091275,weak
19,Business Area Name,0.082476,weak
91,GL Account,0.077232,weak
8,2Y_Past_OA_Difference,0.063989,weak


In [6]:
keep_vars = []
for i, item in enumerate(train_iv_list):
    quantile = train_iv_list[i][1]['IV'].quantile(.6)
    print(quantile)

0.0008404796612658697
0.0010627491984969675
0.0010977555748660985
0.0009975227833041085
0.0018012352478217351
0.002116294820754388
0.0021010129570593313


In [7]:
train_iv_list[i][1].sort_values(ascending=False, by='IV').loc[train_iv_list[i][1]['IV']<=.002][['VAR_NAME','IV']].head()

,VAR_NAME,IV
30,Business Area Name_Health & Human Services,0.001960
22,Business Area Name_City Controller's Office,0.001923
27,Business Area Name_Finance,0.001871
15,3Y_Past_Original_Budget,0.001663
24,Business Area Name_City Secretary,0.001536


In [8]:
keep_vars = []
for i, item in enumerate(train_iv_list):
    year_keep = list(train_iv_list[i][1].sort_values(ascending=False, by='IV').loc[train_iv_list[i][1]['IV']>=.003]['VAR_NAME'])
    if i==0:
        keep_vars = year_keep
        #print(len(year_keep))
    else:
        keep_vars = list(set(year_keep) | set(keep_vars))
        #print(len(year_keep))
        

drop_vars = set(set(list(fy_list[0].columns)).symmetric_difference(keep_vars))
drop_vars

{'Actuals',
 'Astros Playoff Rounds',
 'Astros Yearly Attendance',
 'Business Area Name_Business Opportunity',
 "Business Area Name_City Controller's Office",
 'Business Area Name_City Secretary',
 'Business Area Name_Citywide General Government',
 'Business Area Name_Finance',
 'Business Area Name_General Debt Services',
 'Business Area Name_Housing and Community Devp.',
 'Business Area Name_Houston Information Tech Svcs',
 'Business Area Name_Information Technology (IT)',
 'Business Area Name_Legal Department',
 "Business Area Name_Mayor's Office",
 'Business Area Name_Municipal Court Judicial',
 'Business Area Name_Municipal Courts Department',
 'Business Area Name_Planning & Development',
 'Commitment Set Description',
 'Commitment Set Description_50 - Total Expenditures',
 'Current Budget',
 'Fiscal Year',
 'Fund Name_Asset Forfeiture Fund - Justice',
 'Fund Name_Asset Forfeiture Fund - State',
 'Fund Name_Cable Television Fund',
 'Fund Name_Child Safety Fund',
 'Fund Name_Expansi

In [9]:
pd.DataFrame(pd.Series(keep_vars))

,0
0,GL Account
1,Business Area Name_Houston Emergency Center (HEC)
2,3Y_2Y_1Y_Original_Budget_Mean
3,Fund Name_Building Inspection Fund
4,Business Area Name_City Council
5,Fund Center Name
6,Fund Name_PWE-W & S System Operating Fund
7,Business Area Name_Houston Police Department-HPD
8,GL Category
9,Fund Type Descr_Special Revenue Funds


In [10]:
keep_vars += ['Fiscal Year']

In [11]:
len(keep_vars)

65

In [12]:
woe_pca_df_list = []
woe_pca_explained_var_ratio_list = []

for i, y in enumerate(fy_list):
    woe = WoeEncoder(columns=keep_vars, suffix='')
    target = 'Over Budget'
    y = fy_list[i][target]
    
    hold_df = woe.fit_transform(fy_list[i], y)
    
    pca = PCA(n_components=50)
    
    X = hold_df[keep_columns].fillna(0)
    del hold_df; gc.collect()
    
    woe_pca_df_list.append(pca.fit(X))
    woe_pca_explained_var_ratio_list.append(pca.explained_variance_ratio_.sum())
    

/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:83: RuntimeWarning: divide by zero encountered in log
  d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:84: RuntimeWarning: divide by zero encountered in log
  d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:83: RuntimeWarning: divide by zero encountered in log
  d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:84: RuntimeWarning: divide by zero encountered in log
  d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:83: RuntimeWarning: divide by zero encountered in log
  d3["WO

/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  default = self.iv_df[self.iv_df['VAR_NAME'] == col][self.iv_df['MIN_VALUE'].isna()]['WOE'].mean()
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  default = self.iv_df[self.iv_df['VAR_NAME'] == col][self.iv_df['MIN_VALUE'].isna()]['WOE'].mean()
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  default = self.iv_df[self.iv_df['VAR_NAME'] == col][self.iv_df['MIN_VALUE'].isna()]['WOE'].mean()
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  default = self.iv_df[s

/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  default = self.iv_df[self.iv_df['VAR_NAME'] == col][self.iv_df['MIN_VALUE'].isna()]['WOE'].mean()
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  default = self.iv_df[self.iv_df['VAR_NAME'] == col][self.iv_df['MIN_VALUE'].isna()]['WOE'].mean()
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  default = self.iv_df[self.iv_df['VAR_NAME'] == col][self.iv_df['MIN_VALUE'].isna()]['WOE'].mean()
/Users/josephdown/Documents/Data Science/Data-Projects/Houston Open Data/WOE_Houston.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  default = self.iv_df[s

NameError: name 'keep_columns' is not defined

In [ ]:
#11:16

In [13]:
fy_list[0].head()

,Unnamed: 0,Fund Id,Business Area,Fund Center Id,GL Account,GL Description,GL Category,GL Category Description,Revenue or Expenditure,Fiscal Year,Fund Name,Fund Type Descr,Business Area Name,Fund Center Name,Commitment Set Description,Original Budget,Current Budget,Actuals,3Y_Past_Actuals,2Y_Past_Actuals,1Y_Past_Actuals,2Y_1Y_Actuals_Mean,3Y_2Y_1Y_Actuals_Mean,3Y_Past_Original_Budget,2Y_Past_Original_Budget,1Y_Past_Original_Budget,2Y_1Y_Original_Budget_Mean,3Y_2Y_1Y_Original_Budget_Mean,3Y_Past_OA_Difference,2Y_Past_OA_Difference,1Y_Past_OA_Difference,2Y_1Y_OA_Difference_Mean,3Y_2Y_1Y_OA_Difference_Mean,OA_Difference,Yearly Crime Index,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Rain Fall,Astros Playoff Rounds,Astros Yearly Attendance,Rockets Playoff Rounds,Texans Playoff Rounds,Texans Average Attendance,Rockets Total Attendance,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Houston Police Department,Business Area Name_Human Resources Dept.,Business Area Name_Information Technology (IT),Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Court Judicial,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Business Area Name_Houston Police Department-HPD,Business Area Name_Houston Information Tech Svcs,Over Budget,1YP_OA_Diff_Over_80K
0,0,0.072953,-0.132372,-0.137269,0.030004,0.228927,0.029464,0.029464,Expenditures,0.0,0.072953,0.030989,0.272466,1.661031,50 - Total Expenditures,-0.232945,113579.0,121344.82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7765.82,596.6,80.0,62.0,47.01,0.0,2521076.0,0.0,0.0,71080.0,699280.0,0,0,0.001911,0.003642,-0.005404,0,0,0.000271,0,0,-0.001314,0.072953,0,0.026634,0.004219,0,0.005044,0,0,0.000475,0,-0.001694,0.002699,0,-0.004264,0.020812,0.00208,-0.010404,0,0.004216,0.023087,0.030989,0.004634,-0.001427,-0.007688,0,0,-0.015532,0,0,-0.005024,0

In [ ]:
# #bar = progressbar.ProgressBar()

# hold_list = []
# for split in splits_to_try:
    
#     hold_dict = {}
    
#     X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
#     y_train = X_train['Over Budget']
#     X_train.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)
#     X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
#     y_test = X_test['Over Budget']
#     y_original_budget = X_test['Original Budget']
#     X_test.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)

#     model = xgb.XGBClassifier() #subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
#     model.fit(X_train,y_train)

#     xgb_test_predictions = model.predict(X_test)
#     xgb_train_predictions = model.predict(X_train)

#     model_test_auc = metrics.roc_auc_score(y_test, xgb_test_predictions)
#     model_train_auc = metrics.roc_auc_score(y_train, xgb_train_predictions)
#     original_budget_auc = metrics.roc_auc_score(y_test, y_original_budget)
    
#     hold_dict.update({'split': split[2]})
#     hold_dict.update({'model_test_auc': model_test_auc})
#     hold_dict.update({'model_train_auc': model_train_auc})
#     hold_dict.update({'original_budget_auc': original_budget_auc})
    
    
#     hold_list.append(hold_dict)
    



In [ ]:
# results_df = pd.DataFrame(hold_list)
# results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
# results_df


In [ ]:
# results_df['model_test_auc'].mean()

##adding binary predictor as a feature for XGBReggresor to use later

In [14]:
hold_list = []
fy_augmented = []
for index, split in enumerate(splits_to_try):
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Over Budget']
    X_train.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Over Budget']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBClassifier() #subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)
    
    xgb_test_predictions_probability = [x[1] for x in model.predict_proba(X_test)]
    xgb_train_predictions_probability = [x[1] for x in model.predict_proba(X_train)]

    model_test_auc = metrics.roc_auc_score(y_test, xgb_test_predictions)
    model_train_auc = metrics.roc_auc_score(y_train, xgb_train_predictions)
    original_budget_auc = metrics.roc_auc_score(y_test, y_original_budget)
    
    if index <= 5:
        hold_df = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]]) #grab the test dataframe
        hold_df['Over_Budget_Prediction'] = xgb_test_predictions_probability #add in the over under budget predictions
        fy_augmented.append(hold_df) #store it back in the fy_list
    
    
    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_auc': model_test_auc})
    hold_dict.update({'model_train_auc': model_train_auc})
    hold_dict.update({'original_budget_auc': original_budget_auc})
    hold_dict.update({'model':model})
    
    
    
    hold_list.append(hold_dict)
    

In [15]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
results_df


,model,model_test_auc,model_train_auc,original_budget_auc,split,Model Improvement
0,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.548257,0.559648,0.465611,10-11 to predict 11,0.082646
1,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.661957,0.617208,0.453048,10-11 to predict 12,0.208909
2,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.686382,0.631021,0.458223,10-12 to predict 13,0.228158
3,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.692827,0.694170,0.462134,11-13 to predict 14,0.230693
4,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.694399,0.700737,0.429731,12-14 to predict 15,0.264668
5,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.687940,0.699670,0.464239,13-15 to predict 16,0.223701
6,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.692490,0.649470,0.429731,10-13 to predict 14,0.262759
7,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.691738,0.690002,0.429731,11-14 to predict 15,0.262007
8,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.691079,0.699651,0.464239,12-15 to predict 16,0.226840


In [16]:
for i, y in enumerate(fy_list):
    if i ==0:
        None
    else:
        fy_list[i] = fy_augmented[i-1]

In [17]:
xgb_test_predictions[:10]

array([False,  True, False, False, False, False, False, False, False,
       False])

In [18]:
fy_list[6][['Over_Budget_Prediction','OA_Difference']].head(20)

,Over_Budget_Prediction,OA_Difference
0,0.188154,-39514.09
1,0.759089,0.00
2,0.183639,-140.00
3,0.378538,-7371.38
4,0.217099,-145900.65
5,0.143638,-1036.55
6,0.319938,1763.48
7,0.239981,-11131.33
8,0.066094,-2300.00
9,0.164408,927.00


In [19]:
splits_to_try = [
    
                #predicting off "two years"
                 #[(0,1),(1,2),'10-11 to predict 11'],
                 #[(0,2),(2,3),'10-11 to predict 12'],
                 #[(0,3),(3,4),'10-12 to predict 13'],
                 #[(0,3),(3,5),'10-12 to predict 13 and 14'],
                 [(1,4),(4,5),'11-13 to predict 14'], 
                 #[(1,4),(4,6),'11-13 to predict 14 and 15'],
                 [(2,5),(5,6),'12-14 to predict 15'],
                 #[(2,5),(5,7),'12-14 to predict 15 and 16'],
                 [(3,6),(6,7),'13-15 to predict 16'], 
    
                #predicting off "three" years
    
                 #[(0,4),(5,6),'10-13 to predict 14'],
                 #[(0,4),(4,6),'10-13 to predict 14 and 15'],
                 #[(0,4),(4,7),'10-13 to predict 14 and 15 and 16'],
                 [(1,5),(5,6),'11-14 to predict 15'],
                 #[(1,5),(5,7),'11-14 to predict 15 and 16'],
                 [(2,6),(6,7),'12-15 to predict 16']
                ]
        

#predicting Actuals with binary classifier "over budget" prediction as a predictor

In [21]:
hold_list = []
for split in splits_to_try:
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Actuals']
    X_train.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Actuals']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBRegressor(subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)

    model_test_RMSE = np.sqrt(np.mean((xgb_test_predictions-y_test)**2))
    model_train_RMSE = np.sqrt(np.mean((xgb_train_predictions-y_train)**2))
    original_budget_RMSE = np.sqrt(np.mean((y_original_budget-y_test)**2))

    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_RMSE': model_test_RMSE})
    hold_dict.update({'model_train_RMSE': model_train_RMSE})
    hold_dict.update({'original_budget_RMSE': original_budget_RMSE})
    
    
    hold_list.append(hold_dict)
    

/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[23:19:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:20:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:21:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [22]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['original_budget_RMSE']-results_df['model_test_RMSE']
results_df


,model_test_RMSE,model_train_RMSE,original_budget_RMSE,split,Model Improvement
0,789649.208910,132317.118303,626022.863584,11-13 to predict 14,-163626.345326
1,454009.419063,129515.891054,511970.949010,12-14 to predict 15,57961.529947
2,509428.455007,125910.208660,526290.141141,13-15 to predict 16,16861.686134
3,419603.498950,141711.039563,511970.949010,11-14 to predict 15,92367.450060
4,530016.764038,136810.419675,526290.141141,12-15 to predict 16,-3726.622897


In [27]:
splits_to_try = [
    
                #predicting off "two years"
                 #[(0,1),(1,2),'10-11 to predict 11'],
                 #[(0,2),(2,3),'10-11 to predict 12'],
                 #[(0,3),(3,4),'10-12 to predict 13'],
                 #[(0,3),(3,5),'10-12 to predict 13 and 14'],
                 [(1,4),(4,5),'11-13 to predict 14'], 
                 #[(1,4),(4,6),'11-13 to predict 14 and 15'],
                 [(2,5),(5,6),'12-14 to predict 15'],
                 #[(2,5),(5,7),'12-14 to predict 15 and 16'],
                 [(3,6),(6,7),'13-15 to predict 16'], 
    
                #predicting off "three" years
    
                 #[(0,4),(5,6),'10-13 to predict 14'],
                 #[(0,4),(4,6),'10-13 to predict 14 and 15'],
                 #[(0,4),(4,7),'10-13 to predict 14 and 15 and 16'],
                 [(1,5),(5,6),'11-14 to predict 15'],
                 [(1,5),(5,7),'11-14 to predict 15 and 16'],
                 [(2,6),(6,7),'12-15 to predict 16']
                ]
        

In [36]:
hold_list = []
for split in splits_to_try:
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Actuals']
    X_train.drop(['OA_Difference','Actuals','Current Budget','Over Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Actuals']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Current Budget','Over Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBRegressor(subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)

    model_test_RMSE = np.sqrt(np.mean((xgb_test_predictions-y_test)**2))
    model_train_RMSE = np.sqrt(np.mean((xgb_train_predictions-y_train)**2))
    original_budget_RMSE = np.sqrt(np.mean((y_original_budget-y_test)**2))

    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_RMSE': model_test_RMSE})
    hold_dict.update({'model_train_RMSE': model_train_RMSE})
    hold_dict.update({'original_budget_RMSE': original_budget_RMSE})
    
    
    hold_list.append(hold_dict)
    

[01:03:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:03:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:04:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:04:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:04:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:05:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [37]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['original_budget_RMSE']-results_df['model_test_RMSE']
results_df

,model_test_RMSE,model_train_RMSE,original_budget_RMSE,split,Model Improvement
0,795046.702418,138021.393781,626022.863584,11-13 to predict 14,-169023.838835
1,462089.514065,139063.975894,511970.949010,12-14 to predict 15,49881.434945
2,523543.670406,139532.616466,526290.141141,13-15 to predict 16,2746.470736
3,429268.567660,151057.015301,511970.949010,11-14 to predict 15,82702.381350
4,554844.444561,151057.015301,519179.913562,11-14 to predict 15 and 16,-35664.530999
5,547130.617422,145816.590080,526290.141141,12-15 to predict 16,-20840.476281


# stop here

In [31]:
# sns.set_style('whitegrid')

# x = np.linspace(-1000000000,1000000000,1000000000)
# plt.figure(figsize=(12,12))
# plt.scatter(y_train, xgb_train_predictions, c='red', alpha=.95)
# plt.scatter(y_test, xgb_test_predictions, c='blue', alpha=.95)
# plt.scatter(y_test, y_original_budget, c='green', alpha=.95)
# plt.plot(x,x,c='black')
# plt.xlim([-5,1000000000])
# plt.ylim([-5,1000000000])
# plt.xlabel('observations')
# plt.ylabel('predictions')


In [ ]:
print('hello')

In [38]:
weight_dict = model.get_booster().get_score(importance_type="weight")
weight_df = pd.DataFrame(weight_dict, index=range(len(weight_dict)))
weight_df.T[0].sort_values(ascending=False)

Original Budget                  240
1Y_Past_Actuals                  100
1Y_Past_OA_Difference             61
Unnamed: 0                        51
2Y_Past_Actuals                   48
2Y_Past_OA_Difference             43
2Y_1Y_OA_Difference_Mean          32
Fiscal Year                       29
2Y_1Y_Actuals_Mean                28
3Y_Past_OA_Difference             23
3Y_Past_Actuals                   21
3Y_2Y_1Y_OA_Difference_Mean       20
1Y_Past_Original_Budget           16
1YP_OA_Diff_Over_80K              10
2Y_Past_Original_Budget            9
3Y_Past_Original_Budget            9
Fund Id                            9
GL Account                         7
Over_Budget_Prediction             7
2Y_1Y_Original_Budget_Mean         5
3Y_2Y_1Y_Actuals_Mean              3
Business Area                      3
Yearly Rain Fall                   2
Fund Center Id                     2
3Y_2Y_1Y_Original_Budget_Mean      1
Name: 0, dtype: int64

In [ ]:
sns.set_style('darkgrid')
sns.set(font_scale=2.5)
 
fig, ax = plt.subplots(figsize=(20,15))
bins_ = np.arange(-80000,80000,80000/50)

sns.distplot(model_residuals, label='Model Residuals', norm_hist=True, bins = bins_, color='green',   ax = ax)
sns.distplot(original_budget_residuals, label='Original Budget Residuals',bins=bins_, color='black',  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(-80000,80000)
ax.legend()

In [44]:
#reference: https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier


In [45]:
fy_list[0][drop_list].head(1)

,Fund Name,Fund Type Descr,Business Area Name,Commitment Set Description,GL Description,GL Category Description,Revenue or Expenditure,Fund Type Descr,Fund Center Name
0,0.072953,0.030989,0.272466,50 - Total Expenditures,0.228927,0.029464,Expenditures,0.030989,1.661031


In [65]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


train_df = pd.concat([fy_list[2], fy_list[3], fy_list[4]])
Y = train_df['Actuals'].values
X = train_df.drop(['OA_Difference','Actuals','Current Budget','Over Budget'] + drop_list, axis=1)
test_df = pd.concat([fy_list[5], fy_list[6]])
test = test_df.drop(['OA_Difference','Actuals','Current Budget','Over Budget'] + drop_list, axis=1)
y_test = test_df['Actuals']
original_budget = test_df['Original Budget']

In [47]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [5],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

In [50]:
xgb_model = xgb.XGBRegressor()

#xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
  #                  silent=True, nthread=1)

In [51]:
#ran

In [71]:
fy_list[1].head()

,Unnamed: 0,Fund Id,Business Area,Fund Center Id,GL Account,GL Description,GL Category,GL Category Description,Revenue or Expenditure,Fiscal Year,Fund Name,Fund Type Descr,Business Area Name,Fund Center Name,Commitment Set Description,Original Budget,Current Budget,Actuals,3Y_Past_Actuals,2Y_Past_Actuals,1Y_Past_Actuals,2Y_1Y_Actuals_Mean,3Y_2Y_1Y_Actuals_Mean,3Y_Past_Original_Budget,2Y_Past_Original_Budget,1Y_Past_Original_Budget,2Y_1Y_Original_Budget_Mean,3Y_2Y_1Y_Original_Budget_Mean,3Y_Past_OA_Difference,2Y_Past_OA_Difference,1Y_Past_OA_Difference,2Y_1Y_OA_Difference_Mean,3Y_2Y_1Y_OA_Difference_Mean,OA_Difference,Yearly Crime Index,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Rain Fall,Astros Playoff Rounds,Astros Yearly Attendance,Rockets Playoff Rounds,Texans Playoff Rounds,Texans Average Attendance,Rockets Total Attendance,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Houston Police Department,Business Area Name_Human Resources Dept.,Business Area Name_Information Technology (IT),Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Court Judicial,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Business Area Name_Houston Police Department-HPD,Business Area Name_Houston Information Tech Svcs,Over Budget,1YP_OA_Diff_Over_80K,Over_Budget_Prediction
0,0,1000,3800,3800050001,501070,501070 - Pension - Civilian,500,500 - Personnel Services,Expenditures,2011,General Fund,General Funds,Health & Human Services,HHS-Office Prog Supp,50 - Total Expenditures,103841.0,103841.00,109080.27,999999999999999,999999999999999,121344.82,999999999999999,999999999999999,999999999999999,999999999999999,113579.0,999999999999999,999999999999999,999999999999999,999999999999999,7765.82,999999999999999,999999999999999,5239.27,540.1,79.0,61.0,42.72,0.0,2331490.0,0.0,2.0,71496.0,677658.0,0,0,0,0,0,0,0,0,0,0,

In [66]:
train_df.head()

,Unnamed: 0,Fund Id,Business Area,Fund Center Id,GL Account,GL Description,GL Category,GL Category Description,Revenue or Expenditure,Fiscal Year,Fund Name,Fund Type Descr,Business Area Name,Fund Center Name,Commitment Set Description,Original Budget,Current Budget,Actuals,3Y_Past_Actuals,2Y_Past_Actuals,1Y_Past_Actuals,2Y_1Y_Actuals_Mean,3Y_2Y_1Y_Actuals_Mean,3Y_Past_Original_Budget,2Y_Past_Original_Budget,1Y_Past_Original_Budget,2Y_1Y_Original_Budget_Mean,3Y_2Y_1Y_Original_Budget_Mean,3Y_Past_OA_Difference,2Y_Past_OA_Difference,1Y_Past_OA_Difference,2Y_1Y_OA_Difference_Mean,3Y_2Y_1Y_OA_Difference_Mean,OA_Difference,Yearly Crime Index,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Rain Fall,Astros Playoff Rounds,Astros Yearly Attendance,Rockets Playoff Rounds,Texans Playoff Rounds,Texans Average Attendance,Rockets Total Attendance,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Houston Police Department,Business Area Name_Human Resources Dept.,Business Area Name_Information Technology (IT),Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Court Judicial,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Business Area Name_Houston Police Department-HPD,Business Area Name_Houston Information Tech Svcs,Over Budget,1YP_OA_Diff_Over_80K,Over_Budget_Prediction
0,0,1000,3800,3800050001,501070,501070 - Pension - Civilian,500,500 - Personnel Services,Expenditures,2012,General Fund,General Funds,Health & Human Services,HHS-Office Prog Supp,50 - Total Expenditures,128929.0,128929.0,138553.97,1.000000e+15,121344.82,109080.27,115212.545,1.000000e+15,1.000000e+15,113579.0,103841.0,108710.0,1.000000e+15,1.000000e+15,7765.82,5239.27,6502.545,1.000000e+15,9624.97,534.2,82.0,63.0,24.57,0.0,2067016.0,0.0,2.0,71665.0,663839.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0

In [54]:
clf = GridSearchCV(xgb_model, parameters, n_jobs=5, verbose=2, refit=True)

In [55]:
clf.fit(X, Y)

#trust your CV!
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


XGBoostError: [01:18:52] src/objective/regression_obj.cu:101: label must be in [0,1] for logistic regression
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000001a18875579 dmlc::LogMessageFatal::~LogMessageFatal() + 57
  [bt] (1) 2   libxgboost.dylib                    0x0000001a189051f8 xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 1160
  [bt] (2) 3   libxgboost.dylib                    0x0000001a188717ef xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 1311
  [bt] (3) 4   libxgboost.dylib                    0x0000001a18890314 XGBoosterUpdateOneIter + 180
  [bt] (4) 5   libffi.6.dylib                      0x000000010576b884 ffi_call_unix64 + 76
  [bt] (5) 6   ???                                 0x00007ffeead856b0 0x0 + 140732838467248



In [ ]:
folds = 3
param_comb = 5

#skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, n_jobs=4, verbose=3, random_state=1001 )

#cv=skf.split(X,Y)
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
xgb_test_predictions = random_search.predict(test)
#results_df = pd.DataFrame(data={'Actuals':y_test[:,1]})
#results_df.to_csv('submission-random-grid-search-xgb-porto-01.csv', index=False)


In [ ]:
model_test_RMSE = np.sqrt(np.mean((xgb_test_predictions-y_test)**2))
#model_train_RMSE = np.sqrt(np.mean((xgb_train_predictions-y_train)**2))
original_budget_RMSE = np.sqrt(np.mean((original_budget-y_test)**2))


In [ ]:
y_test.shape


In [ ]:
len(y_original_budget)


In [ ]:
a =pd.Series([7,8,9,10])
b=pd.Series([1,2,1])
np.mean([36,36,64])
np.mean((a-b)**2)
original_budget[:10]


In [ ]:
y_test[:10]

In [ ]:
pd.DataFramey_original_budget[:1000].values



In [ ]:
y_test.std()


In [ ]:
model_test_RMSE


In [ ]:
original_budget_RMSE


In [ ]:
weight_dict = random_search.best_estimator_.get_booster().get_score(importance_type="weight")
weight_df = pd.DataFrame(weight_dict, index=range(len(weight_dict)))
weight_df.T[0].sort_values(ascending=False)


In [ ]:
len(y_original_budget)

In [ ]:
len(xgb_test_predictions)

In [ ]:
len(y_test)

In [ ]:
sns.set_style('whitegrid')

x = np.linspace(-1000000000,1000000000,1000000000)
plt.figure(figsize=(12,12))
#plt.scatter(y_train, xgb_train_predictions, c='red', alpha=.95)
plt.scatter(y_test, xgb_test_predictions, c='blue', alpha=.95)
plt.scatter(y_test, y_original_budget, c='green', alpha=.95)
plt.plot(x,x,c='black')
plt.xlim([-5,1000000000])
plt.ylim([-5,1000000000])
plt.xlabel('observations')
plt.ylabel('predictions')


# H2o

In [56]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,4 days 9 hours 24 mins
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 15 days
H2O cluster name:,H2O_from_python_josephdown_97mll2
H2O cluster total nodes:,1
H2O cluster free memory:,3.475 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [60]:
# create h2o frame for training data
training_frame = train_df
training_frame.columns = list(training_frame.columns[:-1]) + ['Actuals']
training_frame = h2o.H2OFrame(training_frame)

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Found duplicate column name Actuals
  Request: POST /3/Parse
    data: {'destination_frame': 'Key_Frame__upload_b4f052975b79102d857267380fbe4ee5.hex', 'parse_type': 'CSV', 'separator': '44', 'single_quotes': 'False', 'check_header': '1', 'number_columns': '111', 'chunk_size': '4194304', 'delete_on_done': 'True', 'blocking': 'False', 'column_types': '["Numeric","Numeric","Numeric","Numeric","Numeric","Enum","Numeric","Enum","Enum","Numeric","Enum","Enum","Enum","Enum","Enum","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Nume

In [ ]:
# create h2o frame for testing data
testing_frame = pd.concat([X_test,y_test], axis=1)
testing_frame.columns = list(testing_frame.columns[:-1]) + ['Actuals']
testing_frame = h2o.H2OFrame(testing_frame)

In [ ]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(None, 'Actuals', training_frame)

In [ ]:
y_test.std()

In [ ]:
predicted_actuals = aml.predict(h2o.H2OFrame(pd.concat([X_test, y_test], axis=1)))
np.sqrt(np.mean((predicted_actuals['predict'].as_data_frame()['predict'].values-y_test)**2))

In [ ]:
aml.leader

In [ ]:
#3:55 pm

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_train, aml.predict(training_frame).as_data_frame()['predict'].values, c='red')
plt.scatter(y_test, aml.predict(testing_frame).as_data_frame()['predict'].values, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')

In [ ]:
#9/4 4:42pm